In [ ]:
#Operaciones algebraicas
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#filtrado para suavizar los datos
from scipy.signal import savgol_filter

# Forecasting Demanda de Energía (Holt-Winters)

### Leemos los datos que tenemos

In [ ]:
# Importo el archivos de datos de consumo de energia.
df = pd.read_csv(r'Raw_Data/medidor_1.csv')
df.shape

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
df.rename(columns={'fechahora':'DateTime', 'demanda_activa':'y[kW]'}, inplace = True)
df.drop(columns='terminal', inplace=True)
df.head()

<b>Target_values: "y[kW]"</b>

In [ ]:
#Convierto el index en DateTimeIndex
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.sort_values(by=['DateTime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()

## Limpieza de datos

### Eliminación de datos duplicados

In [ ]:
df.shape

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
df.drop_duplicates(subset = 'DateTime', keep = 'last', inplace = True)
df.shape

### Tratando los espacios vacios

In [ ]:
df_2 = df.set_index('DateTime')
# print(f'df.index.freq is set to: {df.index.freq}')
df_2.head()

In [ ]:
df_2.drop(['2017-08-18 09:15:00'], inplace = True)
df_2.head()

In [ ]:
print(df_2.index.min())
print(df_2.index.max())

In [ ]:
print(f'df_2.index.freq is set to: {df_2.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(df_2.index),
                          end = max(df_2.index),
                          freq = '15min') 
#freq = '15min' indica frecuencia por minutos.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}')

In [ ]:
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original
print(data_range.difference(df_2.index))

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (y[kW])
df_3 = df_2.reindex(data_range)

# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
df_3['y[kW]'].interpolate(method='linear', inplace=True)

# Con la interpolación se tiene un datetime (set de hora y dias) continuo
print(f'La df.index.freq ahora es: {df_3.index.freq}, indicando que ya no tenemos valores perdidos')

In [ ]:
df_3['date_and_time'] = df_3.index 
df_3['year'] = df_3.index.year

### Filtro savgol_filter

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_3.index, y=df_3['y[kW]'],
                         mode='lines',
                         name='Datos'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
# Aplica filtro elegido en base a buscar cual mejor se adecua
y_filtered = df_3[["y[kW]"]].apply(savgol_filter,  window_length=5, polyorder=3)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_3.index,y=df_3['y[kW]'],
                         mode='lines',
                         name='No Filtrada'))
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
y_filtered.head()

In [ ]:
y_filtered.info()

## Histograma

In [ ]:
features_num = ["y[kW]"]
y_filtered[features_num].hist(figsize=(10,4)); #el ; es para evitar una grafica duplicada

## EDA (Exploratory data analysis)

### Extraemos características de la variable Tiempo
<i>
Podemos dividir la columna de Datetime en sus diferentes componentes. <br>
Esto nos permite encontrar patrones para diferentes grupos.
</i>

In [ ]:
y_filtered['dow'] = y_filtered.index.day_of_week
y_filtered['doy'] = y_filtered.index.day_of_year
y_filtered['month'] = y_filtered.index.month
y_filtered['year'] = y_filtered.index.year
y_filtered['quarter'] = y_filtered.index.quarter
y_filtered['hour'] = y_filtered.index.hour
y_filtered['minutes'] = y_filtered.index.minute
y_filtered['weekday'] = y_filtered.index.day_name()
y_filtered['woy'] = y_filtered.index.isocalendar().week #week of year
y_filtered['dom'] = y_filtered.index.day # Day of Month
y_filtered['date'] = y_filtered.index.date 

# número de estación del año
y_filtered['season'] = y_filtered['month'].apply(lambda month_number: (month_number%12 + 3)//3) 
# el operador aritmético // solo devuelve a parte entera de la división.

In [ ]:
y_filtered.head()

### Gráficos
<br>
<b>Graficando el consumo de energía a lo largo del tiempo</b>

In [ ]:
#Plotting
fig = px.line(y_filtered, x=y_filtered.index, y='y[kW]', title=f'Demanda kW por tiempo [{min(y_filtered.year)} - {max(y_filtered.year)}]')
fig.update_traces(line=dict(width=0.5))
fig.update_layout(xaxis_title='Date & Time', yaxis_title='Demanda Energía [kW]')
fig.show()

Estudiando la gráfica se observa un comportamiento con patron en temporadas (estación del año). 

### Patrones de fecha y hora. 

Podemos usar nuestras funciones de fecha y hora extraídas previamente <br>
para ver si surgen patrones recurrentes de los datos agregados. <br>
Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = y_filtered.groupby(['hour', 'weekday'], as_index=False).agg({'y[kW]':'median'})
# patron_1.tail()

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'y[kW]', 
              color='weekday', 
              title='Mediana de consumo de energia por hs por día de semana ')

fig.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig.show()

El consumo de energía parece ser más elevados todos los días de la semana excepto los domingos. 

In [ ]:
# Dataframe definido para graficar el consumo horario por temporada del año. Mediana de la energía. 
patron_2 = y_filtered.groupby(['hour', 'season'], as_index=False).agg({'y[kW]':'median'})

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'y[kW]', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_2.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig_2.show()

Durante el verano hay mas consumo????

## Descompoción de la serie de tiempo

Los puntos que representan datos a lo largo de una serie de tiempo pueden ser interesantes <br>
en cuanto sus patrones se complementes con tendencias de subida/bajada y/o estacionalidad. <br>
Según la info adquirida en el EDA esto parece ser así. <br>
<br>
Debido a que la variación por estaciones del año parece constante a lo largo del tiempo, <br>
usaremos el <b> modelo de sumas por descomposición </b> a diferencia del <br>
<b>modelo multiplicativo</b> que es útil para casos donde la variación incrementa en el tiempo.
<br>
Extracto de texto sacado de: __[PennState](https://online.stat.psu.edu/stat510/lesson/5/5.1)__
<br>
<blockquote>
The following two structures are considered for basic decomposition models:
<ol>
<li>Additive:  = Trend + Seasonal + Random</li>
<li>Multiplicative:  = Trend * Seasonal * Random</li>
</ol>
</blockquote>

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# seasonal_decompose necesita un dataframe con un index en formato datatime
series = y_filtered[['y[kW]']] #devuelve un dataframe. Si fuese df_3['demand_in_MW'] devuelve una serie

#Añadido: la season (patrón de movimiento) es anual, con un periodo(fre) igual a la cantidad de hs por año.
frequency = 24*365

# Descomposición de la serie de tiempo, con una freq (hs)= 24 hs por 365 días. 
# El modelo aditivo parace ser el mças acertado dado que no poseo una tendencia de 
# incremento/decremento en la gráfica. 
decomposed = seasonal_decompose(series, model='additive', period=frequency)
#obtengo un objeto con los siguientes atributos: temporada, tendencia y residuo:
# ------------- atributos = [[decomposed.seasonal], [decomposed.trend], [decomposed.resid]] -------------

In [ ]:
df_4 = y_filtered.copy()

In [ ]:
# df_trend = pd.DataFrame(decomposed.trend)
# df_seasonal = pd.DataFrame(decomposed.seasonal)
# df_resid = pd.DataFrame(decomposed.resid)
# df_4 = pd.concat([df_3, df_trend, df_seasonal, df_resid], axis=0)
df_4['Trend'] = decomposed.trend
df_4['Seasonal'] = decomposed.seasonal
df_4['Residuos'] = decomposed.resid

In [ ]:
fig_t = px.line(df_4,
                y = 'Trend',
                title='Trend')

# adjust line width
fig_t.update_traces(line=dict(width=2))
        
# change layout of axes and the figure's margins 
# to emulate tight_layout
fig_t.update_layout(
                xaxis=dict(showticklabels=False, linewidth=1),
                yaxis=dict(title=''),
                margin=go.layout.Margin(l=40, r=40, b=0, t=40, pad=0)
)

fig_t.show()

In [ ]:
fig_s = px.line(df_4,
                y = 'Seasonal',
                title='Seasonality')

# adjust line width
fig_s.update_traces(line=dict(width=0.25))
        
# change layout of axes and the figure's margins 
# to emulate tight_layout
fig_s.update_layout(
                xaxis=dict(showticklabels=False, linewidth=1),
                yaxis=dict(title=''),
                margin=go.layout.Margin(l=40, r=40, b=0, t=40, pad=0)
)

fig_s.show()

In [ ]:
fig_r = px.line(df_4,
                y = 'Residuos',
                title='Residuos')

# adjust line width
fig_r.update_traces(line=dict(width=0.5))
        
# change layout of axes and the figure's margins 
# to emulate tight_layout
fig_r.update_layout(
                xaxis=dict(showticklabels=False, linewidth=1),
                yaxis=dict(title=''),
                margin=go.layout.Margin(l=40, r=40, b=0, t=40, pad=0)
)

fig_r.show()

# Modelos de estimación (Forecasting)

Se tendrán en cuenta los siguientes modelos:
    <ol>
    <li> Triple Exponential Smoothing: Holt-Winters </li>
    <li> Explicit Multi-Seasonality: Prophet </li>
    </ol>

## Train/Test
<b>Objetivo</b>: Estimación precisa de la demanda de los proximos 12 meses. <br>
Para lograr el objetivo, restringimos los datos de "entrenamiento" a algunos años antes de la fecha max.<br>
Esto lo hacemos para evitar tendencias que jodan al modelo (por lo analizado, no parece ser el caso)

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(df_3.index)}')
print(f'El último punto de medicion fecha/hs es: {max(df_3.index)}')

In [ ]:
# Dataframe de recort
CUTOFF_DATE = pd.to_datetime('2021-10-04')
# TIME_DELTA = pd.DateOffset(years=3)

# Separo df p/ test y df p/ train
train = df_3.loc[(df_3.index < CUTOFF_DATE)].copy() #& (df_3.index >= CUTOFF_DATE - TIME_DELTA)].copy()
test = df_3.loc[df_3.index >= CUTOFF_DATE].copy()

In [ ]:
#Se permite recortar varias fechas porque:
#1- El comportamiento es constante en el tiempo.
#2- Alivia la carga de procesamiento en la PC.
print(f'Training shape: {train.shape}\n Testing shape: {test.shape}\n')
print(f'Las fechas de entrenamiento son: {min(train.index)} & {max(train.index)}')
print(f'Las fechas de test son: {min(test.index)} & {max(test.index)}')